# `app.py`

While `main_window.py`'s responsibility is to **define the graphics user interface**, `app.py`'s responsibility is to **define the functionalities of the GUI**. This is achieved by doing 2 things:

1. Defining `functions` to accomplish certain actions
2. Connecting `Widget` actions to these `functions`

For example, if we want the `Update Window Button` to plot the stock prices in the GUI's canvas. We will have to create a `function` that plots the graph into the canvas and then connect the `Update Window Button` to this `function`.

However, before doing so, `app.py` must first know the `Widget names` defined in `main_window.py`. 

For example, the `Update Window Button` is actually named: `updateWindowButton`. This name is defined on the previous section, when `main_window.ui` was designed using `Qt Designer` and the `objectName` is specified inside the Property Editor!

This is why, on the previous step, it is **recommended** to name the `Widgets` **accordingly!**

This section of the report will go through the 3 steps of developing `app.py` + 1 optional step to compile `app.exe`, as summarized in the graphics below.

![app.py development process](asset/img/app-process.png)

## Inheriting `Widgets` from `main_window.py` 

The goal of this section is to ensure that `app.py` is **runnable without any error** and shows the **exact same** GUI as if previewing `main_window.ui`.

![app.py running correctly](asset/img/gui.png)

This result shows that `app.py` has successfully inherited all the properties of `main_window.py`, which includes all the `Widgets` defined when `main_window.ui` was created! These `Widgets` include `updateWindowButton`, `SMA1Checkbox`, `filePathEdit`, etc...

To achieve this, simply start from the generic starter code for all `PyQt5` application and then add the following:

1. Import `matplotlib`, `PyQt5` and the GUI's Widget class called `UI_Form` from `main_window`
2. Pass `QWidget` and `UI_Form` as argument to `Main` class to specify inheritance from `QWidget` and `UI_Form` class
3. Call the superclass' (`UI_Form`) initializing function and setup function
4. Finally, after the inherited GUI has been initialized, add the `canvas` and `toolbar` widget to the `canvasLayout`

This is exactly shown in the code below, running them should result in the image shown above:

In [ ]:
import sys
from pathlib import Path
from datetime import datetime

# Step 1
# standard matplotlib import statements
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# import matplotlib backend for Qt5
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar

# standard PyQt5 import statements
from PyQt5 import QtCore as qtc
from PyQt5 import QtWidgets as qtw

# importing the class to be inherited from
from main_window import Ui_Form

# importing StockData processing module
from stock_data import StockData

class Main(qtw.QWidget, Ui_Form): # Step 2
    def __init__(self):
        # Step 3
        # calling Ui_Form's initializing and setup function
        super().__init__()
        self.setupUi(self)
        self.setWindowTitle("Stock Chart & Moving Average Application")

        # Step 4
        # sets up figure to plot on, instantiates canvas and toolbar
        self.figure, self.ax = plt.subplots()
        self.canvas = FigureCanvas(self.figure)
        self.toolbar = NavigationToolbar(self.canvas, self)

        # attaches the toolbar and canvas to the canvas layout
        self.canvasLayout.addWidget(self.toolbar)
        self.canvasLayout.addWidget(self.canvas)

if __name__ == "__main__":
    app = qtw.QApplication([])
    main = Main()
    main.show()
    sys.exit(app.exec_())

> *Learning Point: Inheriting `Widgets` from `main_window.py`*

> *When `main_window.ui` is converted into `main_window.py` using `pyuic5`, the `Widget` class called `Ui_Form` is created.*
> *This `Ui_Form` class has access to all the `Widgets` previously defined inside `main_window.ui` using `Qt Designer`!*
> *They're accessible to `Ui_Form` as regular python `Attributes`. e.g: `self.updateWindowButton`, etc...*
> *Thus, by inheriting from `Ui_Form`, `app.py`'s `Main` class can also access these `Widgets` through its `Attributes`.*
> *LIkewise, `functions` defined in `Ui_Form` are also inherited and accessible to `Main`*.

> *Learning Point: Defining & Adding `Widgets` programmatically*

> *Sometimes, it is more convenient to define `Widgets` programmatically then through `Qt Designer`.*
> *As shown from the code snippet above, this is also possible and uses the **exact same core principles** as in `main_window.py`*
> 1. *Defining each `Widget` objects' and their names within the GUI.*
> *Exemplified with lines such as:*
> *`self.canvas = FigureCanvas(self.figure)` or similar instantiation line: `button = QPushButton('Button Name', self)`*
> 2. *Defining the location, size and other physical attributes of each `Widgets`.*
> *Exemplified with lines such as:*
> *`self.canvasLayout.addWidget(self.canvas)`*

Now that `app.py` is able to access the `Widgets` defined in `main_window.py` by means of Python inheritance. It is now possible to implement `app.py`'s main responsibility:

> 1. Defining `functions` to accomplish certain actions
> 2. Connecting `Widget` actions to these `functions`

## Defining `functions` in `app.py`

Before defining the `functions` in `app.py`, it is important to first be aware of the scope of each `functions` needed to execute the app's entire process. By referring to the User Manual's 5-step guide, it is possible to breakdown the entire app's functionalities into 3 major functions + 2 minor functions:

1. `load_data(self)` : invoked when `Load CSV File Button` is pressed

> loads stock data .csv from inputted filepath string on the GUI as StockData object,
> also autocompletes all inputs using information provided by the csv.
> (Handles the actions from Step 1-2 of User Manual).

2. `update_canvas(self)` : invoked when `Load Update Window Button` is pressed

> creates a datetime object from the inputted date string of format YYYY-MM-DD.
> uses it to slice a copy of loaded stock_data to be used to update graphics.
> checks checkboxes first to see if SMA1, SMA2, Buy and Sell plots need to be drawn.
> finally, updates graphic accordingly.
> (Handles the actions from Step 3-5 of User Manual).

3. `plot_graph(self, column_headers, formats)` : invoked when `update_canvas function` is called

> plots graphs specified under columnd_headers using the formats specified
> (Helps to handle the action from Step 5 of User Manual).

4. `report(self, string)` : invoked when any of the 3 major functions are called

> given a report (string), update the scroll area with this report

5. `center(self)` : invoked `__init__(self)` is called (i.e. during the startup of app)

> centers the fixed main window size according to user screen size

In [53]:
import sys
sys.path.insert(1, '../src')

from app import Main
import inspect # standard library used later to get info about the source code

def beautify(code): # prints 'line code' with 2 less indent and without the def header
    codeline = lambda code, start : [(start + 1 + i, code[i]) for i in range(len(code))]
    print("".join([f"{line} {text[2:]}" if len(text) > 1 else f"{line} {text}" for line, text in codeline(code[0][1:], code[1])]))

### `load_data(self)`

`loda_data` is relatively simple function. 

First and foremost,

In [54]:
beautify(inspect.getsourcelines(Main.load_data))

91 """
92 loads stock data .csv from inputted filepath string on the GUI
93 as StockData object, also autocompletes all inputs
94 using information provided by the csv.
95 
96 Error handling
97 	invalid filepath :
98 		empty filepath or file could not be found.
99 	invalid .csv :
100 		.csv file is empty, missing date column, etc.
101 """
102 filepath = Path(self.filePathEdit.text())
103 
104 try:
105 	self.stock_data = StockData(filepath)
106 	start_date, end_date = self.stock_data.get_period()
107 	period = f"{start_date} to {end_date}"
108 
109 	# auto-complete feauture
110 	self.startDateEdit.setText(start_date)
111 	self.endDateEdit.setText(end_date)
112 	self.periodEdit.setText(period)
113 	self.SMA1Edit.setText("15")
114 	self.SMA2Edit.setText("50")
115 	self.SMA1Checkbox.setChecked(False)
116 	self.SMA2Checkbox.setChecked(False)
117 
118 	self.report(f"Data loaded from {filepath}; period auto-selected: {start_date} to {end_date}.")
119 	print(self.stock_data.data)
120 
121 exce

### `update_canvas(self)`

In [55]:
beautify(inspect.getsourcelines(Main.update_canvas))

128 """
129 creates a datetime object from the inputted date string
130 of format YYYY-MM-DD. uses it to slice a copy of loaded
131 stock_data to be used to update graphics. checks
132 checkboxes first to see if SMA1, SMA2, Buy and Sell plots
133 need to be drawn. finally, updates graphic accordingly.
134 
135 Error handling
136 invalid date format:
137 	date format inside the .csv file is not YYYY-MM-DD
138 non-existent stock_data :
139 	the selected range results in an empty dataframe
140 	or end date < start date
141 non-existent data point :
142 	data of that date does not exist,
143 	or maybe because it is Out-Of-Bound
144 raised exceptions :
145 	SMA1 and SMA2 values are the same,
146 	or other exceptions raised
147 """
148 self.ax.clear()
149 self.date_format = '%Y-%m-%d'
150 
151 try:
152 	start_date = str(datetime.strptime(self.startDateEdit.text(), self.date_format).date())
153 	end_date = str(datetime.strptime(self.endDateEdit.text(), self.date_format).date())
154 	period = 

### `plot_graph(self, column_headers, formats)`

In [56]:
beautify(inspect.getsourcelines(Main.plot_graph))

195 """
196 plots graphs specified under columnd_headers using the formats
197 
198 Parameters
199 column_headers : [str, str, ...]
200 	a list containing column header names with data to be plotted
201 formats : [str, str, ...]
202 	a list of matplotlib built-in style strings to indicate
203 	whether to plot line or scatterplot and the colours
204 	corresponding to each value in col_headers
205 	(hence, must be same length)
206 
207 Error handling
208 empty dataframe :
209 	selected dataframe is empty
210 """
211 self.ax.clear()
212 assert not self.selected_stock_data.empty
213 
214 # matplotlib has its own internal representation of datetime
215 # date2num converts datetime.datetime to this internal representation
216 x_data = list(mdates.date2num(
217                               [datetime.strptime(dates, self.date_format).date()
218                               for dates in self.selected_stock_data.index.values]
219                               ))
220 
221 colors = ['black', 'bl

### `report(self, string)`

In [57]:
beautify(inspect.getsourcelines(Main.report))

243 """
244 given a report (string), update the scroll area with this report
245 
246 Parameters
247 string : str
248 	string of the report, usually the error message itself.
249 """
250 report_text = qtw.QLabel(string)
251 self.scrollLayout.addWidget(report_text)
252 print(string)



### `center(self)`

In [60]:
beautify(inspect.getsourcelines(Main.center))

255 """
256 centers the fixed main window size according to user screen size
257 """
258 screen = qtw.QDesktopWidget().screenGeometry()
259 main_window = self.geometry()
260 x = (screen.width() - main_window.width()) / 2
261 y = (screen.height() - main_window.height()) / 2 - 50	# pulls the window up slightly (arbitrary)
262 self.setFixedSize(main_window.width(), main_window.height())
263 self.move(x, y)



## Connecting `Widget` actions to `functions`

blabla

In [59]:
beautify(inspect.getsourcelines(Main.__init__))

59 """
60 initializes and sets up GUI widgets and its connections
61 """
62 super().__init__()
63 self.setupUi(self)
64 self.setWindowTitle("Stock Chart & Moving Average Application")
65 
66 # sets up figure to plot on, instantiates canvas and toolbar
67 self.figure, self.ax = plt.subplots()
68 self.canvas = FigureCanvas(self.figure)
69 self.toolbar = NavigationToolbar(self.canvas, self)
70 
71 # attaches the toolbar and canvas to the canvas layout
72 self.canvasLayout.addWidget(self.toolbar)
73 self.canvasLayout.addWidget(self.canvas)
74 
75 # sets up a scroll area to display GUI statuses
76 self.scrollWidget = qtw.QWidget()
77 self.scrollLayout = qtw.QVBoxLayout()
78 self.scrollWidget.setLayout(self.scrollLayout)
79 self.scrollArea.setWidget(self.scrollWidget)
80 
81 # button & checkbox connections
82 self.loadCSVButton.clicked.connect(self.load_data)
83 self.updateWindowButton.clicked.connect(self.update_canvas)
84 self.SMA1Checkbox.stateChanged.connect(self.update_canvas)
85 self.S

## (Optional) Compiling `app.exe`